**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 2 - Open Science Platform - User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

In [28]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [29]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.

In [30]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-02-09 10:19:47,568 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-02-09 10:19:47.569 | INFO     | __main__:<module>:3 - b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'


2 - Creates a user account into the blockchain.

In [31]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_user_account(create_account_contract_address, user_account_short_id, DOMAIN, user_public_key, user_account)

2025-02-09 10:19:47.663 | INFO     | __main__:<module>:13 - User Role: publisher
2025-02-09 10:19:47.665 | INFO     | __main__:<module>:24 - Creating account with name: unruffled_merkle
2025-02-09 10:19:47.667 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-09 10:19:47.686 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-09 10:19:47.687 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 10:19:47.688 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-09 10:19:47.689 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"
2025-02-09 10:19:47,705 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 10:19:47,707 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 

2025-02-09 10:19:50,597 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 10:19:50,605 - INFO - ('COMMITTED', 5, 0)
2025-02-09 10:19:50.608 | INFO     | dump_to_json:dump_to_json_ld:82 - Appended new entry to file 'datasets/accounts.json', current total entries: 2
2025-02-09 10:19:50.609 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "create_user_account"


3 - Queries the blockchain to confirm the succesful creation of the account.

In [32]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-02-09 10:19:50.626 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-09 10:19:50,629 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 10:19:50,647 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 10:19:50,649 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 10:19:53,628 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 10:19:53,636 - INFO - ('COMMITTED', 5, 0)
2025-02-09 10:19:53,638 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '7b30a443d6f2d0ec861114a73335735de5f93623f4ac7a57a5830bbb80da3cfa', 'timestamp': 1739096393638}
2025-02-09 10:19:53.640 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-09 10:19:53,643 - INFO - Appended data under 'unruffled_merkle@test' in logs/account_data.json
2025-02-09 10:19:53.644 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-09 1

4 - Loads the smart contract for details (attributes) setting into the Iroha 1 blockchain .

In [33]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-09 10:19:53,686 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-09 10:19:53.687 | INFO     | __main__:<module>:3 - b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [34]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

2025-02-09 10:19:53.743 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-09 10:19:53.765 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'
2025-02-09 10:19:53.767 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: A518E3EE4D45C20EF834361093305D2500A46228
2025-02-09 10:19:53.768 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-09 10:19:53.770 | DEBUG    | integration_helpers:tracer:35 - 	Entering "process_account"
2025-02-09 10:19:53,772 - INFO - Processing account: unruffled_merkle@test
2025-02-09 10:19:53,773 - INFO - User Account Metadata: {'@type': 'foaf:Person', 'foaf:name': 'Unruffled Merkle', 'foaf:mbox': 'unruffled_merkle@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Lebanese American University'}, 

2025-02-09 10:19:56,655 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 10:19:56,664 - INFO - ('COMMITTED', 5, 0)
2025-02-09 10:19:56.665 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-09 10:19:56.688 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001
2025-02-09 10:19:56.689 | DEBUG    | integration_helpers:get_engine_receipts_result:140 - Result decoded string:                                                                                                
2025-02-09 10:19:56.690 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_result"
2025-02-09 10:19:56.692 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "set_account_detail"
2025-02-09 10:19:56.694 | DEBUG    | integration_helpers

6 - Queries the blockchain again to confirm the succesfull details setting.

In [35]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid = process_raw_data(details)

2025-02-09 10:19:56.711 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-09 10:19:56,714 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 10:19:56,732 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 10:19:56,734 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 10:19:59,687 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 10:19:59,696 - INFO - ('COMMITTED', 5, 0)
2025-02-09 10:19:59,698 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '0d1e8d1e0096c1e84ea6f13933b0e7bda9e9264a62eb8d4a4f4eb318987b787c', 'timestamp': 1739096399698}
2025-02-09 10:19:59.699 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-09 10:19:59,702 - INFO - Appended data under 'unruffled_merkle@test' in logs/account_data.json
2025-02-09 10:19:59.704 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-09 1

7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [36]:
user_metadata = download_json_from_ipfs(account_metadata_cid)
logger.info(f"User Metadata: {user_metadata}")

2025-02-09 10:19:59.757 | INFO     | __main__:<module>:2 - User Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'unruffled_merkle@test', 'schema:publicKey': '578f3ce2d29080f483bd3bb80aea86df59eebdaf4701aa6bff21290e5676eee1', 'schema:roleName': 'publisher'}, 'foaf:mbox': 'unruffled_merkle@email.com', 'foaf:name': 'Unruffled Merkle', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Lebanese American University'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '7882-5876-3606-X'}}
